In [1]:
#Import Dependencies
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import time

In [2]:
#Import data
accident_df=pd.read_csv("Resources/accident-data.csv")
accident_df.head()

,accident_index,accident_year,accident_reference,longitude,latitude,accident_severity,number_of_vehicles,number_of_casualties,date,day_of_week,...,second_road_class,second_road_number,pedestrian_crossing_human_control,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area
0,2020010219808,2020,10219808,-0.254001,51.462262,3,1,1,04/02/2020,3,...,6,0,9,9,1,9,9,0,0,1
1,2020010220496,2020,10220496,-0.139253,51.470327,3,1,2,27/04/2020,2,...,6,0,0,4,1,1,1,0,0,1
2,2020010228005,2020,10228005,-0.178719,51.529614,3,1,1,01/01/2020,4,...,6,0,0,0,4,1,2,0,0,1
3,2020010228006,2020,10228006,-0.001683,51.541210,2,1,1,01/01/2020,4,...,6,0,0,4,4,1,1,0,0,1
4,2020010228011,2020,10228011,-0.137592,51.515704,3,1,2,01/01/2020,4,...,5,0,0,0,4,1,1,0,0,1


In [3]:
accident_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91199 entries, 0 to 91198
Data columns (total 27 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   accident_index                           91199 non-null  object 
 1   accident_year                            91199 non-null  int64  
 2   accident_reference                       91199 non-null  object 
 3   longitude                                91185 non-null  float64
 4   latitude                                 91185 non-null  float64
 5   accident_severity                        91199 non-null  int64  
 6   number_of_vehicles                       91199 non-null  int64  
 7   number_of_casualties                     91199 non-null  int64  
 8   date                                     91199 non-null  object 
 9   day_of_week                              91199 non-null  int64  
 10  time                                     91199

In [4]:
#Drop NA values
dropped=accident_df.dropna()

In [5]:
#Drop unnecessary columns
cleaned = dropped.drop(columns=["accident_year","accident_reference","first_road_class","first_road_number",\
                                   "road_type","junction_detail","junction_control","second_road_class","second_road_number",\
                                   "pedestrian_crossing_human_control","pedestrian_crossing_physical_facilities",\
                                   "special_conditions_at_site","carriageway_hazards","light_conditions","weather_conditions",\
                                   "number_of_vehicles"])

In [6]:
#Drop values with unknown or missing data
unknown_road=cleaned[cleaned["road_surface_conditions"] ==9]
cleaned.drop(unknown_road.index[unknown_road['road_surface_conditions'] == 9], inplace=True)
missingdata=cleaned[cleaned["road_surface_conditions"] ==-1]
cleaned.drop(missingdata.index[missingdata['road_surface_conditions'] == -1], inplace=True)
speedlimit=cleaned[cleaned["speed_limit"] ==-1]
cleaned.drop(speedlimit.index[speedlimit['speed_limit'] == -1], inplace=True)
unallocated=cleaned[cleaned["urban_or_rural_area"] ==3]
cleaned.drop(unallocated.index[unallocated['urban_or_rural_area'] == 3], inplace=True)
missingurban=cleaned[cleaned["urban_or_rural_area"] == -1]
cleaned.drop(missingurban.index[missingurban['urban_or_rural_area'] == -1], inplace=True)

In [7]:
#Adding descriptions for severity
cleaned.loc[cleaned["accident_severity"] ==3,"severity"] = "Slight"
cleaned.loc[cleaned["accident_severity"] ==2,"severity"] = "Serious"
cleaned.loc[cleaned["accident_severity"] ==1,"severity"] = "Fatal"

In [8]:
#Adding descriptions for road surface condition
cleaned.loc[cleaned["road_surface_conditions"] ==1,"road_surface"] = "Dry"
cleaned.loc[cleaned["road_surface_conditions"] ==2,"road_surface"] = "Wet or damp"
cleaned.loc[cleaned["road_surface_conditions"] ==3,"road_surface"] = "Snow"
cleaned.loc[cleaned["road_surface_conditions"] ==4,"road_surface"] = "Frost or ice"
cleaned.loc[cleaned["road_surface_conditions"] ==5,"road_surface"] = "Flood over 3cm. deep"
cleaned.loc[cleaned["road_surface_conditions"] ==6,"road_surface"] = "Oil or diesel"
cleaned.loc[cleaned["road_surface_conditions"] ==7,"road_surface"] = "Mud"
cleaned.loc[cleaned["road_surface_conditions"] ==-1,"road_surface"] = "Data missing or out of range"
cleaned.loc[cleaned["road_surface_conditions"] ==9,"road_surface"] = "unknown (self reported)"

In [9]:
#Adding descriptions for day of the week
cleaned.loc[cleaned["day_of_week"] ==1,"Weekday"] = "Sunday"
cleaned.loc[cleaned["day_of_week"] ==2,"Weekday"] = "Monday"
cleaned.loc[cleaned["day_of_week"] ==3,"Weekday"] = "Tuesday"
cleaned.loc[cleaned["day_of_week"] ==4,"Weekday"] = "Wednesday"
cleaned.loc[cleaned["day_of_week"] ==5,"Weekday"] = "Thursday"
cleaned.loc[cleaned["day_of_week"] ==6,"Weekday"] = "Friday"
cleaned.loc[cleaned["day_of_week"] ==7,"Weekday"] = "Saturday"

In [10]:
#Grouping days of the week into weekend or weekday
cleaned.loc[cleaned["day_of_week"] ==1,"daytype"] = "Weekend"
cleaned.loc[cleaned["day_of_week"] ==2,"daytype"] = "Workday"
cleaned.loc[cleaned["day_of_week"] ==3,"daytype"] = "Workday"
cleaned.loc[cleaned["day_of_week"] ==4,"daytype"] = "Workday"
cleaned.loc[cleaned["day_of_week"] ==5,"daytype"] = "Workday"
cleaned.loc[cleaned["day_of_week"] ==6,"daytype"] = "Workday"
cleaned.loc[cleaned["day_of_week"] ==7,"daytype"] = "Weekend"

In [11]:
#Grouping time by the hour
cleaned['time'] = pd.to_datetime(cleaned['time'])
cleaned['Time by the Hour'] = cleaned['time'].dt.floor('H')
cleaned['Time by the Hour'] = [time.time() for time in cleaned['Time by the Hour']]
cleaned['time'] = [time.time() for time in cleaned['time']]

In [12]:
#Grouping rush hours together, 6:00–9:59 and 15:00–18:59
 
cleaned.loc[cleaned["Time by the Hour"] == time(0, 0, 0),"Block_of_Time"] = "Night"
cleaned.loc[cleaned["Time by the Hour"] == time(1, 0, 0),"Block_of_Time"] = "Late Night"
cleaned.loc[cleaned["Time by the Hour"] == time(2, 0, 0),"Block_of_Time"] = "Late Night"
cleaned.loc[cleaned["Time by the Hour"] == time(3, 0, 0),"Block_of_Time"] = "Late Night"
cleaned.loc[cleaned["Time by the Hour"] == time(4, 0, 0),"Block_of_Time"] = "Late Night"
cleaned.loc[cleaned["Time by the Hour"] == time(5, 0, 0),"Block_of_Time"] = "Early Day"
cleaned.loc[cleaned["Time by the Hour"] == time(6, 0, 0),"Block_of_Time"] = "Morning Rush Hour"
cleaned.loc[cleaned["Time by the Hour"] == time(7, 0, 0),"Block_of_Time"] = "Morning Rush Hour"
cleaned.loc[cleaned["Time by the Hour"] == time(8, 0, 0),"Block_of_Time"] = "Morning Rush Hour"
cleaned.loc[cleaned["Time by the Hour"] == time(9, 0, 0),"Block_of_Time"] = "Morning Rush Hour"
cleaned.loc[cleaned["Time by the Hour"] == time(10, 0, 0),"Block_of_Time"] = "Early Day"
cleaned.loc[cleaned["Time by the Hour"] == time(11, 0, 0),"Block_of_Time"] = "Early Day"
cleaned.loc[cleaned["Time by the Hour"] == time(12, 0, 0),"Block_of_Time"] = "Early Day"
cleaned.loc[cleaned["Time by the Hour"] == time(13, 0, 0),"Block_of_Time"] = "Day"
cleaned.loc[cleaned["Time by the Hour"] == time(14, 0, 0),"Block_of_Time"] = "Day"
cleaned.loc[cleaned["Time by the Hour"] == time(15, 0, 0),"Block_of_Time"] = "Evening Rush Hour"
cleaned.loc[cleaned["Time by the Hour"] == time(16, 0, 0),"Block_of_Time"] = "Evening Rush Hour"
cleaned.loc[cleaned["Time by the Hour"] == time(17, 0, 0),"Block_of_Time"] = "Evening Rush Hour"
cleaned.loc[cleaned["Time by the Hour"] == time(18, 0, 0),"Block_of_Time"] = "Evening Rush Hour"
cleaned.loc[cleaned["Time by the Hour"] == time(19, 0, 0),"Block_of_Time"] = "Day"
cleaned.loc[cleaned["Time by the Hour"] == time(20, 0, 0),"Block_of_Time"] = "Day"
cleaned.loc[cleaned["Time by the Hour"] == time(21, 0, 0),"Block_of_Time"] = "Night"
cleaned.loc[cleaned["Time by the Hour"] == time(22, 0, 0),"Block_of_Time"] = "Night"
cleaned.loc[cleaned["Time by the Hour"] == time(23, 0, 0),"Block_of_Time"] = "Night"

In [13]:
#Final Cleaned Data
cleaned

,accident_index,longitude,latitude,accident_severity,number_of_casualties,date,day_of_week,time,speed_limit,road_surface_conditions,urban_or_rural_area,severity,road_surface,Weekday,daytype,Time by the Hour,Block_of_Time
1,2020010220496,-0.139253,51.470327,3,2,27/04/2020,2,13:55:00,20,1,1,Slight,Dry,Monday,Workday,13:00:00,Day
2,2020010228005,-0.178719,51.529614,3,1,01/01/2020,4,01:25:00,30,2,1,Slight,Wet or damp,Wednesday,Workday,01:00:00,Late Night
3,2020010228006,-0.001683,51.541210,2,1,01/01/2020,4,01:50:00,30,1,1,Serious,Dry,Wednesday,Workday,01:00:00,Late Night
4,2020010228011,-0.137592,51.515704,3,2,01/01/2020,4,02:25:00,30,1,1,Slight,Dry,Wednesday,Workday,02:00:00,Late Night
5,2020010228012,-0.025880,51.476278,3,1,01/01/2020,4,01:30:00,20,1,1,Slight,Dry,Wednesday,Workday,01:00:00,Late Night
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91194,2020991027064,-2.926320,56.473539,2,1,12/08/2020,4,14:30:00,30,1,1,Serious,Dry,Wednesday,Workday,14:00:00,Day
91195,2020991029573,-4.267565,55.802353,3,1,13/11/2020,6,15:05:00,30,1,1,Slight,Dry,Friday,Workday,15:00:00,Evening Rush Hour
91196,2020991030297,-2.271903,57.186317,2,1,15/04/2020,4,12:42:00,60,1,2,Serious,Dry,Wednesday,Workday,12:00:00,Early Day
91197,2020991030900,-3.968753,55.950940,3,1,15/12/2020,3,14:00:00,30,1,1,Slight,Dry,Tuesday,Workday,14:00:00,Day


In [14]:
# Exporting cleaned data to resources folder to then be used in the data analysis file
cleaned.to_csv("resources/final_data_analysis_df.csv",index=False)